<a href="https://colab.research.google.com/github/thad75/TP_ENSEA_ELEVE/blob/main/SIA/TP2/Introduction_to_Pytorch_Lightning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pytorch-lightning

# Pytorch-Lightning : Training made Easier

Time : 30 min 

In the last lab, we used Pytorch and Tensorflow to perform classification.
As you have seen, writing the training and testing loop can be quickly indigest. One can get easily lost.

Let us introduce you Pytorch Lightning

<img src="https://warehouse-camo.ingress.cmh1.psfhosted.org/a88de56e65d2ea6bc203ce178a1cecbe9b50a0ac/68747470733a2f2f6769746875622e636f6d2f5079546f7263684c696768746e696e672f7079746f7263682d6c696768746e696e672f7261772f312e342e392f646f63732f736f757263652f5f7374617469632f696d616765732f6c6f676f2e706e67">


Pytorch lightning will handle a lot of things for you. It creates a Trainer which is a Code Management trick used by many companies (FB, Google..) in order to get much more digest code.


More Information on : https://www.pytorchlightning.ai/

Goal of this lab :

* Use Pytorch Lightning for Training
* Learn to use Trainers and write comprehensible code in Class
* Do classification on MNIST


# Classify Numbers using Lightning

This introduction should be quick as is it just reapplying what you've learned in the previous lab.

What's interesting about Lightning is that you can plug in your Torch modules without any modification.

# LightningDataModule : MNIST

As you have seen in the previous lab, you need to create your dataset and then load it into a DataLoader. That Dataloader is then used to fetch and give data as input for your Model.

You will see that using Lightning makes things clearer. LightningDataModule allows you to write cleaner Code and fit easily your data to your model.

First load MNIST Dataset. You won't need to write the Dataset as it's already embedded in Pytorch

In [ ]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torchvision.datasets import MNIST
from torchvision import transforms
from pytorch_lightning import loggers as pl_loggers
import pytorch_lightning as pl

In fact,you can also write the normal PyTorch Dataloader code. But you'll see that writing thit DataModule helps for the training 

# Model : Classifier 

We are performing Classification on the MNIST Dataset. You know that MNIST is given in torchvision. We will need to download and load it in ou LightningDataModule class.

Have a look at : https://pytorch.org/vision/stable/datasets.html

In [ ]:
class MNISTDataModule(pl.LightningDataModule):

    def __init__(self):
        super().__init__()
        self.transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
        self.data_dir = ''
        self.batch_size_train, self.batch_size_valid, self.batch_size_test = 32,32,32

    def prepare_data(self):
        # download MNIST Dataset from torchvision, this method is used to transform or download data to your Google Colab Session 
        MNIST(self.data_dir, train=True, download=True)
        #...

    def setup(self, stage):
        #We need to setup our module. We have a training set that we will be fitting in our model
        #and a testing set used to test our models prediction.
        #the stage variable corresponds to those two steps : 
        #         |fit
        # stage = <test
        #         |None  

        #First stage is 'fit' (or None)
        if stage == "fit" or stage is None:

            mnist_dataset = MNIST(self.data_dir, train=True, transform=self.transform)
            # What additional set can we create ? Why
            self.mnist_train, _ = 

        #Second stage is 'test' 
        if stage == "test" or stage is None:

            self.mnist_test = MNIST(self.data_dir, train=False, transform=self.transform)
            #What additional set can we create ? Why ?

    def train_dataloader(self):
        #Now create your Training DataLoader
        

    def val_dataloader(self):
        #Now create your Validation DataLoader
        
    def test_dataloader(self):
        #Now create your Testing DataLoader
        


As you can see, we added a drop_last variable in our DataLoader:
* What is its purpose ?
* Is it useful ?

# LightningModule :  MNIST Classifier 

Design a model to perform Classification. Again, ask yourself the following questions: 
* What task is it ?
* What data do I have ?
* What learning rate should I use ?
* What could be my loss ? Why ?
* What non-linearity should I use ?

In [ ]:
class MNISTClassifier(pl.LightningModule):
    def __init__(self, output_shape):
        super(MNISTClassifier,self).__init__()
        # what is the output_shape of your model ?
        self.output_shape = output_shape
        self.save_hyperparameters()
        # Define your model here, be careful, your model will be an instance of the class


    def forward(self,x):
        # What would be the forward steps of this classifier ?

        return x

    def configure_optimizers(self):
        # Choose your optimizer : https://pytorch.org/docs/stable/optim.html
        
        return optimizer

    def training_step(self, train_batch, batch_idx):
        # Define your Training Step
        # This method is pretty much similar to what your did in the first lab to train your model.
        x,y = train_batch     
        ...
        loss = 
        # Don't remove the next line, you will understand why later
        self.log('train_loss', loss)
        return loss


    def validation_step(self, val_batch, batch_idx):
        # Define your Validation Step
        # What is the difference between the Training and the Validation Step ?
        x,y = val_batch
        ...
        loss = 
        # Don't remove the next line, you will understand why later

        self.log('val_loss', loss)
    
    def test_step(self, val_batch, batch_idx):
        # Define your Test Step
        # What is the difference between the Training, Validation and Test Step ?
        x,y = val_batch
        ...
        loss = F.cross_entropy(latent, y)
        # Don't remove the next line, you will understand why later
        self.log('test_loss', loss)

# Did you say Train ?

Let's train the model. We hav to create a Trainer that will handle a lot of thing for us. Lightning trainer is full of interesting assets that helps you for your training.

To get a glance of what Lightning Trainer can give :
https://pytorch-lightning.readthedocs.io/en/latest/common/trainer.html

We also use TensorBoard 

In [ ]:
tb_logger = pl_loggers.TensorBoardLogger("introduction to Lightning")

dm = MNISTDataModule()
model = MNISTClassifier(10)

trainer = pl.Trainer(gpus=-1,max_epochs=10,accelerator='dp',logger=tb_logger)
trainer.fit(model, dm)


Oh it's training ! Happy ? Easy ? Let's test the model

# Did you say Test ?

For testing, well it's pretty easy 

In [ ]:
trainer.test(model)

In this case we were computing the Loss. However testing a model is not based on its lost but other things. We won't ask you to compute them now but in the next part of the lab.

# TensorBoard

TensorBoard is a really useful tool. Indeed, it let's you register interesting values during training and plot them INTERACTIVELY. You might have seen a self.log line in the Validation and Training steps. 
The self.log saves the loss value into a TensorBoard readable file. We can also add images or other values using self.log

In fact, look at the checkpoint created by the training. You might see 3 files :
* Checkpoint
* event.out....
* hparam.yaml

Let's open tensorboard to see how the training was. Tensorboard is loadable using magic_python commands.
More info on TensorBoard : https://www.tensorflow.org/tensorboard/get_started



In [ ]:
%reload_ext tensorboard
%tensorboard --logdir "/content/introduction to Lightning/default/version_0"

Pytorch Lightning can be used along PyTorch. We encourage you to use PyTorch Lightning during your Lab Sessions and Career as it simplifies a lot of things for you (MultiGPU, Learning Rate Decay...)
Have a good time with it

<img src='https://c.tenor.com/VyApQ-jWyV0AAAAC/happy-borat.gif'>

